In [1]:
import pandas as pd
from phono3py.file_IO import write_fc2_to_hdf5, write_fc3_to_hdf5, read_fc3_from_hdf5, read_fc2_from_hdf5

In [3]:
aln_fc3 = read_fc3_from_hdf5(
    "../aln_fcs/fc3.hdf5",
)

aln_fc2 = read_fc2_from_hdf5(
    "../aln_fcs/fc2.hdf5",
)
aln_fc2.shape, aln_fc3.shape

((576, 576, 3, 3), (72, 72, 72, 3, 3, 3))

In [4]:
gan_fc3 = read_fc3_from_hdf5(
    "../gan_fcs/fc3.hdf5",
)

gan_fc2 = read_fc2_from_hdf5(
    "../gan_fcs/fc2.hdf5",
)
gan_fc2.shape, gan_fc3.shape

((576, 576, 3, 3), (72, 72, 72, 3, 3, 3))

In [6]:
al_fraction = [0.1, 0.3, 0.5, 0.7, 0.9]

for fraction in al_fraction:
    fc2_avg = aln_fc2 * fraction + gan_fc2 * (1 - fraction)
    fc3_avg = aln_fc3 * fraction + gan_fc3 * (1 - fraction)
    write_fc3_to_hdf5(fc3_avg, filename=f'fc3_{fraction:.1f}.hdf5')
    write_fc2_to_hdf5(fc2_avg, filename=f'fc2_{fraction:.1f}.hdf5')

In [9]:
def average_lines(line1, line2, fraction):
    vals1 = list(map(float, line1.strip().split()))
    vals2 = list(map(float, line2.strip().split()))
    avg = [a * fraction + b * (1-fraction)  for a, b in zip(vals1, vals2)]
    return " ".join(f"{val:.6f}" for val in avg) + "\n"

al_fraction = [0.1, 0.3, 0.5, 0.7, 0.9]

for fraction in al_fraction:
    with open("../aln_fcs/AlN_BORN", "r") as f1, open("../gan_fcs/GaN_BORN", "r") as f2, open(f"al{fraction}_BORN", "w") as fout:
        first_line = f1.readline()
        __ = f2.readline()
        fout.write(first_line)
        for _ in range(3):
            line1 = f1.readline()
            line2 = f2.readline()
            averaged_line = average_lines(line1, line2, fraction)
            fout.write(averaged_line)
